In [2]:
!hostname

from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig
import transformers
import torch
torch.set_grad_enabled(False)

path="/gpfs/share/home/2201110046/LLM/llama_hf/7b"

tokenizer = LlamaTokenizer.from_pretrained(path, padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

model = LlamaForCausalLM.from_pretrained(path, device_map="cuda")

gpua800n09


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/gpfs/share/home/2201110046/.conda/llama/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.65s/it]


In [17]:
import importlib
from tools import inference
importlib.reload(inference)
from tools import utils
importlib.reload(utils)
from tools import patch_scope
importlib.reload(patch_scope)

<module 'tools.patch_scope' from '/gpfs/share/home/2201110046/patch-scope-reproduce/tools/patch_scope.py'>

In [28]:
from tools.utils import extract_hidden, break2tokens
from tools.patch_scope import patch_scope
from tools.inference import forward

prompts=["Amazon's former CEO attended Oscars"]
print(break2tokens(tokenizer,prompts))
outputs=forward(model, tokenizer, prompts, with_hiddens=True)
hiddens=extract_hidden(outputs, token_pos=6, layer_id=7)

result=patch_scope(model, tokenizer, hiddens, 
    target_prompt = 'cat->cat; 135->135; hello->hello; ?->', 
    target_token_id = -2,
    target_layer_id = 7,
    gen_len = 2)
print(result)

[['<s>', 'Amazon', "'", 's', 'former', 'CE', 'O', 'attended', 'O', 'sc', 'ars']]
['hello;']


In [30]:
prompts=["lemon->yellow, grape->purple, apples->"]
print(break2tokens(tokenizer,prompts))
outputs=forward(model, tokenizer, prompts, with_hiddens=True)
hiddens=extract_hidden(outputs, token_pos=12, layer_id=7)

result=patch_scope(model, tokenizer, hiddens, 
    target_prompt = 'cat->cat; 135->135; hello->hello; ?->', 
    target_token_id = -2,
    target_layer_id = 7,
    gen_len = 1)
print(result)

[['<s>', 'le', 'mon', '->', 'yellow', ',', 'gra', 'pe', '->', 'pur', 'ple', ',', 'app', 'les', '->']]
['app;']
